بسم الله الرحمن الرحيم


Deuxième étape: On a réussi à prendre les livres de fiqh maliki, maintenant on va essayer de mettre l'auteur...

## Première étape : aller chercher les urls de chacun des livres

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'products')
#s = s.find_all('div', class_ = 'products product_list row grid')
#lines = s.find('h3', class_ = 's_title_block flex_child')
#Affichage du contenu de la page
#print(lines)

#Extraire les liens
urls = []
for line in s.findAll('a'):
    urls.append(line.get('href'))#line.text.split(\'n') est une liste de hcaine de caractères, donc [0] est 
    #seulement le str.
    #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n

Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.

In [2]:
#Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
urls = list(np.unique(urls))
#urls

On va supprimer les liens qui ne renvoient pas vers des livres : 
 - Ceux qui finissent pas '.asc' ou 'desc
 - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
 - les "javascript"

In [3]:
#Suppression des liens ne renvoyant pas vers des livres.
urls.remove('javascript:')
urls.remove('javascript:;')
urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')
to_supp = []
for link in range(len(urls)):
    if urls[link].endswith('asc') or urls[link].endswith('.desc') or urls[link].startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?'):
        to_supp.append(urls[link])
    else:
        to_supp.append('')
    

In [4]:
urls = pd.DataFrame(urls)
A = urls == pd.DataFrame(to_supp) 
urls = urls[A == False].dropna()

In [5]:
urls = urls.values.T.tolist()[0]

In [6]:
urls

['https://www.sifatusafwa.com/fr/fiqh-ahkam/charh-bidayah-al-mujtahid-wa-nihayah-al-muqtasid-abdullah-al-abadi.html',
 'https://www.sifatusafwa.com/fr/fiqh-ahkam/tabsirah-al-hukkam-fi-usul-al-aqdhiyyah-ibn-farhun-799h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-jami-li-masa-il-al-mudawwanah-ibn-yunus-as-siqilli-451h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-mou-amalat-fi-al-fiqh-al-maliki-sadiq-al-ghariani.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-muqaddimah-al-qurtubiyyah-bi-charh-cheikh-zarrouk-el-bernoussi.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-usul-fi-ma-rifah-al-usul-risalah-fi-usul-ad-din-ibn-al-arabi.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/charh-aqidah-malik-as-saghir-al-qayrawani-al-qadi-abdelwahhab.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/dalil-as-salik-ala-as-hal-al-massalik-houssain-as-soulaymani.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/hachiyah-ad-dousouqi-ala-ach-charh-al-kabir-fiqh-mali

On va maintenant essayer de regrouper les données des liens dans un dataframe.

## Essai pour le premier lien

In [7]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))

In [8]:
#L_string

On essaye d'unifier pour enlever les deux "" et afficher en colonnes:

In [9]:
df = pd.DataFrame(L_string)
df = df.drop_duplicates() #des fois les volumes apparaissent deux fois
df.head() #Pour avoir un aperçu

,0,1,2,3
0,,Titre,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...,
1,,Auteur,د. عبد الله العبادي,
2,,Volumes,4,
3,,Pages,2328,
4,,Édition,دار السلام,


In [10]:
df.columns = ['supp', 'categories', 'test', 'supp2']
df.drop(['supp', 'supp2'], axis = 1, inplace=True)

In [11]:
df.set_index('categories', inplace = True)
df.head()

,test
categories,
Titre,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...
Auteur,د. عبد الله العبادي
Volumes,4
Pages,2328
Édition,دار السلام


الحمد لله, il faut maintenant transposer ce tableau et le faire tourner pour tous les sites.

## Tentative de concaténation de deux dataframes

On a pas réussi à concaténer les deux transposés, on va essayer de les concaténer et de les transposer après.

In [12]:
url = urls[1]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df2 = pd.DataFrame((L_string))
df2 = df2.drop_duplicates()
df2 #Pour avoir un aperçu

,0,1,2,3
0,,Titre,تبصرة الحكام في أصول الأقضية ومناهج الأحكام,
1,,Auteur,ابن فرحون المالكي,
2,,Volumes,3,
3,,Pages,2013,
4,,Édition,دار القلم,
5,,Couverture,Rigide,
6,,Vérification et préparation,د. عثمان ضميرية,
8,,Format,17x24cm,
9,,Harakat,En Majorité ou en Totalité,


In [13]:
df2.columns = ['supp', 'categories', 'test', 'supp2']
df2.drop(['supp', 'supp2'], axis = 1, inplace=True)
df2.set_index('categories', inplace = True)
df2 #pour comparer avec df

,test
categories,
Titre,تبصرة الحكام في أصول الأقضية ومناهج الأحكام
Auteur,ابن فرحون المالكي
Volumes,3
Pages,2013
Édition,دار القلم
Couverture,Rigide
Vérification et préparation,د. عثمان ضميرية
Format,17x24cm
Harakat,En Majorité ou en Totalité


In [14]:
M = df.join(on='categories', other = df2, rsuffix='_df2', how = 'outer')

In [15]:
M.set_index(M['categories'], inplace = True)#Si il y a un tahqiq (authentification), une colonne supplémentaire apparaît,
#en faisant ca on dit que l'index sera la colonne "categories" avec le plus de champs remplis.
M.drop(['categories'], axis = 1, inplace = True)

In [16]:
#M

In [17]:
#M.T

Maintenant on peut essayer de faire une boucles sur toutes les pages. On pourra renommer les "categories" par le site où on pourra trouver ce livre.

## Essai de boucle sur tous les sites d'une page donnée

On fait juste le premier lien pour qu'on puisse fusionner sur un DataFrame, on le supprimera juste après inchAllah

In [18]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df = pd.DataFrame((L_string))
df = df.drop_duplicates()#des fois les volumes apparaissent deux fois
df.columns = ['supp', 'categories', str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
df.drop(['supp', 'supp2'], axis = 1, inplace=True)
df.set_index('categories', inplace = True)

In [19]:
urls.remove(urls[0]) #On enlève le lien qu'on a déjà mis dans le dataframe de base

Changements dans la boucle:
 - df2 devient df_other
 - on ajoute le site en colonne

In [20]:
print(urls.index(urls[0]))

0


In [42]:
dfs = []
i = 0
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    for text in s.findAll(class_ ='data-sheet flex_container'):
        L_string.append(text.text.split('\n'))
    L_string = np.array(L_string)
    df_other = pd.DataFrame((L_string))
    df_other = df_other.drop_duplicates()#des fois les volumes apparaissent deux fois
    df_other.columns = ['supp', 'categories',str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
    df_other.drop(['supp', 'supp2'], axis = 1, inplace=True)
    df_other.set_index('categories', inplace = True)
    dfs.append(df_other)

In [43]:
M = pd.concat((d for d in dfs), axis = 1)

In [45]:
M.T

,Titre,Auteur,Volumes,Pages,Édition,Couverture,Vérification et préparation,Format,Harakat,Préface
https://www.sifatusafwa.com/fr/fiqh-ahkam/tabsirah-al-hukkam-fi-usul-al-aqdhiyyah-ibn-farhun-799h.html,تبصرة الحكام في أصول الأقضية ومناهج الأحكام,ابن فرحون المالكي,3,2013,دار القلم,Rigide,د. عثمان ضميرية,17x24cm,En Majorité ou en Totalité,NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-jami-li-masa-il-al-mudawwanah-ibn-yunus-as-siqilli-451h.html,الجامع لمسائل المدونة المسمى مصحف المذهب المالكي,ابن يونس الصقلي,10,5520,إبداع - دار ابن الجوزي القاهرة,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-mou-amalat-fi-al-fiqh-al-maliki-sadiq-al-ghariani.html,المعاملات في الفقه المالكي,الصادق بن عبد الرحمن الغرياني,NaN,333,دار ابن حزم,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-muqaddimah-al-qurtubiyyah-bi-charh-cheikh-zarrouk-el-bernoussi.html,شرح المقدمة القرطبية ليحيى القرطبي,الشيخ أحمد زروق البرنسي,NaN,334,دار ابن حزم,Rigide,د. أحسن زقور,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-usul-fi-ma-rifah-al-usul-risalah-fi-usul-ad-din-ibn-al-arabi.html,الوصول إلى معرفة الأصول,الإمام أبو بكر ابن العربي المالكي,NaN,172,دار الفتح,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/charh-aqidah-malik-as-saghir-al-qayrawani-al-qadi-abdelwahhab.html,شرح عقيدة مالك الصغير ابن أبي زيد القيرواني,القاضي عبد الوهاب بن نصر البغدادي,NaN,344,الرابطة المحمدية للعلماء في المغرب,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/dalil-as-salik-ala-as-hal-al-massalik-houssain-as-soulaymani.html,دليل السالك على أسهل المسالك,لحسن بن محمد سليماني,NaN,613,دار ابن حزم,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/hachiyah-ad-dousouqi-ala-ach-charh-al-kabir-fiqh-maliki.html,حاشية العلامة الدسوقي على الشرح الكبير للإمام ...,العلامة الدسوقي,15,NaN,دار ابن حزم,Rigide,أحمد بن علي الدمياطي,17x24cm,En Majorité ou en Totalité,NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/lawami-al-durar-fi-hatk-astar-al-mukhtasar-al-majlisi-ash-shinqiti.html,لوامع الدرر في هتك أستار المختصر,محمد المجلسي الشنقيطي,14,9032,إبداع - دار ابن الجوزي القاهرة,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/le-statut-du-qabd-dans-la-prière-chez-l-imam-malik-al-mekki-ibn-azouz.html,هيئة الناسك في أن القبض في الصلاة هو مذهب الإم...,محمد المكي بن عزوز,NaN,176,دار الإمام مسلم,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN


الحمد لله Maintenant on peut essayer de généraliser à toutes les pages

## Généralisation à toutes les pages.

J'ai pris le même modèle de code que dans sifatusafwa-1.ipynb. Ici on va préparer la liste des urls.

In [244]:
##Aller chercher les urls des livres
urls = []
to_supp =[]
NB_MAX_PAGES = 15
for i in range(1,NB_MAX_PAGES):
    i = str(i)
    r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/?page=' + i)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id = 'products')
    #s = s.find_all('div', class_ = 'products product_list row grid')
    #lines = s.find('h3', class_ = 's_title_block flex_child')
    #Affichage du contenu de la page
    #print(lines)

    #Extraire les liens
    
    for line in s.findAll('a'):
        urls.append(line.get('href'))#line.text.split(\'n') est une liste de hcaine de caractères, donc [0] est 
        #seulement le str.
        #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n
    #Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
    urls = list(np.unique(urls))
    #urls
    # On va supprimer les liens qui ne renvoient pas vers des livres : 
    #  - Ceux qui finissent pas '.asc' ou 'desc
    #  - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
    #  - les "javascript"
    #Suppression des liens ne renvoyant pas vers des livres.
    for link in range(len(urls)):
        if (urls[link].endswith('asc')) or (urls[link].endswith('.desc')) or (urls[link].startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?')):
            to_supp.append(urls[link])
        #else:
        if link == 28:
            print(urls[link],urls[link].endswith('.desc') )
            
try:
    urls.remove('javascript:')
    urls.remove('javascript:;')
    urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')
except ValueError:
    pass
urls = pd.DataFrame(urls)
#pour qu'on puisse comparer deux DataFrame avec la meme taille:
for i in range(len(to_supp)-len(urls)):
    urls = urls.append(dict(zip(urls.columns,['col'])), ignore_index=True)

#On compare avec les liens à supprimer
A = urls == pd.DataFrame(to_supp) 
urls = urls[A == False].dropna() #ce qui n'est pas False est NaN
urls = urls.values.T.tolist()[0]
#on enlève tout ce qu'on a rajouté maintenant qu'on a pu faire la comparaison avec ce qu'il y avait
#a supprimer
urls = list(np.unique(urls))
urls.remove('col')

https://www.sifatusafwa.com/fr/fiqh-maliki/?rewrite=fiqh-maliki&page=12 False
https://www.sifatusafwa.com/fr/fiqh-maliki/?rewrite=fiqh-maliki&order=product.position.asc False
https://www.sifatusafwa.com/fr/fiqh-maliki/?rewrite=fiqh-maliki&order=product.name.desc True
https://www.sifatusafwa.com/fr/fiqh-maliki/?rewrite=fiqh-maliki False
https://www.sifatusafwa.com/fr/fiqh-maliki/?page=6 False
https://www.sifatusafwa.com/fr/fiqh-maliki/?page=12 False
https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.quantity.desc True
https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.position.asc False
https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.date_add.desc True
https://www.sifatusafwa.com/fr/fiqh-maliki/-amal-ahl-al-madinah-inda-al-malikiyyah.html False
https://www.sifatusafwa.com/fr/fiqh-maliki/-amal-ahl-al-madinah-inda-al-malikiyyah.html False
https://www.sifatusafwa.com/fr/fiqh-maliki/-amal-ahl-al-madinah-inda-al-malikiyyah.html False
https://www.sifatusafwa.com/fr/fiq

In [245]:
urls[28]

'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.date_add.desc'

A ce stade, `urls` devrait contenir les adresses de chacun des livres, pour toutes les pages de fiqh maliki.

Ici, on va construire les dataframes pour chaque page. 
Une page a plusieurs livres. On aura une seule liste qui contiendra toutes les livres de la page 1, ceux de la page2... et on concatènera tout inchAllah

In [205]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df = pd.DataFrame((L_string))
df = df.drop_duplicates()#des fois les volumes apparaissent deux fois
df.columns = ['supp', 'categories', str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
df.drop(['supp', 'supp2'], axis = 1, inplace=True)
df.set_index('categories', inplace = True)
urls.remove(urls[0]) #On enlève le lien qu'on a déjà mis dans le dataframe de base


0


In [207]:
dfs = []
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    for text in s.findAll(class_ ='data-sheet flex_container'):
        L_string.append(text.text.split('\n'))
    L_string = np.array(L_string)
    df_other = pd.DataFrame((L_string))
    df_other = df_other.drop_duplicates()#des fois les volumes apparaissent deux fois
    df_other.columns = ['supp', 'categories',str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
    df_other.drop(['supp', 'supp2'], axis = 1, inplace=True)
    df_other.set_index('categories', inplace = True)
    dfs.append(df_other)
M = pd.concat((d for d in dfs), axis = 1)
M.T

AttributeError: 'NoneType' object has no attribute 'findAll'

In [221]:
pd.DataFrame(to_supp) == pd.DataFrame(urls)

ValueError: Can only compare identically-labeled DataFrame objects